In [2]:
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import warnings     
warnings.filterwarnings(action='ignore')    
import user



In [4]:
#1. mysqldb 접속객체 세팅(연결하기)
connect_datas = {
    'host': user.host,
    'user': user.user,
    'passwd': user.pw,
    'db': user.db,
    'charset': 'utf8'
}
db = MySQLdb.connect(**connect_datas) 


In [6]:
#2. 주가데이터 csv파일 불러오기
news = pd.read_csv('../datas/investing_news.csv')
investing_cr = pd.read_csv('../datas/crawling_investing.csv')

In [11]:
news.columns = ['symbol', 'title', 'link', 'date']

In [14]:
news.drop(columns='date', inplace=True)

In [22]:
news.iloc[0]['link'] = 'https://kr.investing.comhttps://invst.ly/uob92'

In [38]:
import re
for i in range(len(news)):
    try:
        news.iloc[i]['link'] = re.sub('([\S]+)https://', 'https://', news.iloc[i]["link"])
    except :
        pass

In [39]:
news

,symbol,title,link
0,MMM,'신월여의지하도로' 대형차 잇단 끼임 사고…16일부터 과태료 부과,https://invst.ly/uob92
1,MMM,경제 캘린더 - 이번 주 주목해야 할 주요 이슈 5가지,https://kr.investing.com/news/economy/article-...
2,MMM,역사를 바꾼 30초·3m…인저뉴어티 비행 성공 막전막후,https://invst.ly/uithf
3,MMM,“3M-코카콜라 등 73개 기업서 청년인턴 뽑아요”,https://invst.ly/ud0-o
4,MMM,"오늘의 주요 3가지 이슈: 애플, 페이스북, 연준 금리",https://kr.investing.com/news/stock-market-new...
...,...,...,...
1625,YUM,오늘 주목해야 할 시장의 주요 이슈 3가지,https://kr.investing.com/news/stock-market-new...
1626,ZBH,"짐머 바이오메트 Q1 실적, 수익 예상치를 넘어",https://kr.investing.com/news/article-631149
1627,ZION,(BUZZ)美 은행주 하락...브렉시트에 美 금리인상 가능성 희박해져,https://kr.investing.com/news/economy-news/(bu...
1628,ZTS,미국서 오랑우탄도 백신 맞아…동물용 첫 접종,https://invst.ly/u1f4d


In [56]:
# # MYsqldb로 테이블 생성하기
sql = '''CREATE TABLE News ( 
symbol varchar(5) NOT NULL, 
title varchar(200), 
link varchar(200)
) 
''' 
# # cursor객체 생성후 쿼리 실행
curs = db.cursor()
curs.execute(sql)

0

In [44]:
investing_cr[investing_cr['symbol'] == 'AAPL']

,symbol,update_date,querter_sales,profit,operating,real_profit,sales,shared,next_update_date
180,AAPL,210327.0,89584.0,38079.0,27503.0,23630.0,3.254100e+11,1.668763e+10,210802.0
181,AAPL,201226.0,111439.0,44328.0,33534.0,28755.0,3.254100e+11,1.668763e+10,210802.0
182,AAPL,200926.0,64698.0,24689.0,14775.0,12673.0,3.254100e+11,1.668763e+10,210802.0
183,AAPL,200627.0,59685.0,22680.0,13091.0,11253.0,3.254100e+11,1.668763e+10,210802.0


In [46]:
# # MYsqldb로 테이블 생성하기
sql = '''CREATE TABLE Company_low ( 
symbol varchar(5) NOT NULL PRIMARY KEY, 
update_date date, 
querter_sales int(6),
profit int(5),
operating int(5),
real_profit int(5),
sales int(11),
share int(10),
next_update_date date
) 
''' 
# # cursor객체 생성후 쿼리 실행
curs = db.cursor()
curs.execute(sql)

0

In [43]:
investing_cr2

,update_date,sales,profit,operating,real_profit
symbol,,,,,
MMM,210331.0,8851.0,4327.0,1983.0,1624.0
MMM,201231.0,8583.0,4246.0,1839.0,1389.0
MMM,200930.0,8350.0,4047.0,1909.0,1413.0
MMM,NaN,NaN,NaN,NaN,NaN
ABT,210331.0,10456.0,6056.0,2109.0,1793.0
...,...,...,...,...,...
ZION,200630.0,712.0,NaN,NaN,66.0
ZTS,210331.0,1871.0,1328.0,687.0,559.0
ZTS,201231.0,1807.0,1209.0,420.0,359.0


In [47]:
#3. sqlalchemy 클라이언트 설정
client = create_engine(f'mysql://{user.user}:{user.pw}@{user.host}/{user.db}?charset=utf8',encoding="utf-8")
conn = client.connect()

In [94]:
#4. csv파일 db에 담기
main.to_sql(name='main',con=client,if_exists='append',index=False)
conn.close()

In [57]:
news.to_sql(name='News',con=client,if_exists='append',index=False)
investing_cr.to_sql(name='Company_low',con=client,if_exists='append',index=False)

In [55]:
len(news['link'][139])

64